In [16]:
import os
import re
from tqdm import tqdm
import spacy
from spacy.tokens import Doc
from spacy import displacy

In [17]:
nlp = spacy.load("ru_core_news_md")
doc = Doc(nlp.vocab).from_disk("articles_clean_vectors.spacy")

## Грамматические омонимы

Будем искать слова различных частей речи. Также исключим иностранные и с тегами `NUM` и `PROPN`. С PROPN очень много неверно размеченных слов, опять-таки из-за аббревиатур.

In [4]:
from collections import defaultdict

In [54]:
data = defaultdict(lambda: defaultdict(set))

for token in tqdm(doc):
    if token.is_stop or not token.is_alpha:
        continue
    if token.morph.get("Foreign") == ["Yes"]:
        continue
    if token.pos_ in ["PROPN", "NUM"]:
        continue
    data[token.text][token.pos_].add(token)

100%|██████████| 4524841/4524841 [00:06<00:00, 694309.78it/s]


In [55]:
limit = 10
filename = "examples/grammar_hom.txt"
with open(filename, "w") as f:
    for word, pos_data in data.items():
        if len(pos_data) < 2:
            continue
        print(f"Word: {word}", file=f)
        
        for pos, tokens in pos_data.items():
            print(f"{word} ({pos})", file=f)
            for i, token in enumerate(tokens):
                if i >= limit:
                    break
                print(token.sent.text, file=f)
        print(file=f)

В файле `examples/grammar_hom.txt` лежат примеры слов. Но большая часть -- неправильные. В основном из-за неправильно определенной части речи.

Немного вручную отобранных примеров в `examples/grammar_hom_manual.txt`.

## Эллипсис

In [5]:
ellipsis_phrases = []
for sent in tqdm(doc.sents):
    for token in sent:
        if token.dep_ == "conj" and token.head.dep_ == "ROOT":
            ellipsis_phrases.append(sent)

print(len(ellipsis_phrases))

203622it [00:01, 130977.44it/s]

65881


In [6]:
filename = "examples/ellipsis.txt"

with open(filename, "w") as f:
    for sent in ellipsis_phrases:
        print(sent.text, file=f)

In [7]:
for i in range(10):
    print(ellipsis_phrases[i])

С другой стороны, и цены Realme 6/Pro — как у индийских флагманов.
Так что на новинки молодого китайского бренда стоит обратить самое пристальное внимание, и в первую очередь — на самую технически мощную модель, Realme 6 Pro.
Корпус имеет совсем простую обтекаемую форму, плоской рамки по боковому периметру нет, по сути это единый пластиковый кожух, склеенный со стеклом экрана.
Если надеть комплектный гибкий темный чехол, то удерживаться смартфон будет еще лучше, но в габаритах прибавит изрядно.
Камеры сильно выступают за пределы поверхности, поэтому на столе смартфон лежит неустойчиво.
Любопытно, что даже не будучи истинным флагманом, новинка получила продвинутую двойную камеру спереди.
Правда, цвет проявляется лишь в градиентном узоре на задней панели, весь остальной корпус, по сути, черный.
Ну и, как обычно, до нас самые яркие цвета почему-то не доходят, оранжевый на российский рынок не поставляется.
Физические размеры экрана составляют 69×153 мм, соотношение сторон — 20:9, плотность

Весь вывод в `examples/ellipsis.txt`.

Здесь уже большая часть предложений определена верно.

## Лексические омонимы

Код дальше создаст файлы с предложениями и словами. Эти файлы будут использоваться в другом ноутубке (`lexical_hom.ipynb`).

In [18]:
cnt_sent = 5000
filename_sents = "lexical/sents.txt"
filename_words = "lexical/words.txt"

words = defaultdict(int)
for sent in list(doc.sents)[:cnt_sent]:
    for token in sent:
        if not token.is_alpha or token.is_stop:
            continue
        if token.morph.get("Foreign") == ["Yes"]:
            continue
        if token.pos_ in ["PROPN"]:
            continue
        
        words[token.text.lower()] += 1

In [19]:
with open(filename_sents, "w") as f:
    for sent in list(doc.sents)[:cnt_sent]:
        print(sent.text.lower(), file=f)
        
threshold = 3
with open(filename_words, "w") as f:
    for word, cnt in words.items():
        if cnt > 5:
            print(word, file=f)

## Референциальные цепочки

Тут возникли большие трудности. Действительно долго искал и не смог найти работающей модели для русского языка. Для английского неплохие общедоступные модели существуют уже несколько лет. А для русского языка модели, которые я нашел, просто не получилось запустить. 

Я так понял, что относительно недавно появился размеченный корпус RuCoCo для задачи coreference resolution, и у некоторых библиотек (Spacy, в частности) в экспериментальных функциях есть заготовки для обучения моделей для распознавания цепочек. Так что в ближайшее время может появиться модели с более простым интерфейсом.